# Functional Demostration - with MicroPython Remote

[![SigmaDSP ESP32 Control Test](https://raw.githubusercontent.com/Wei1234c/SigmaDSP/master/jpgs/demo_upy2.jpg)](https://youtu.be/R3jcmxOhpao)     

In [1]:
# %local
# !ls /dev/ttyUSB*

No serial connected
  %connect to connect
  %lsmagic to list commands

In [1]:
# %connect /dev/ttyUSB0 --baudrate=115200 --user='micro' --password='python' --wait=0

%connect COM6 --baudrate=115200 --user='micro' --password='python' --wait=0

Connected on COM6


In [1]:
import gc
import os
import sys 

#  for ESP32 =========================== 
import peripherals
import adapters
from adau1401 import ADAU1401 as ADAU
from ufactory import Factory


def collect_garbage():
    gc.collect()
    if sys.platform == 'esp32':
        print('[Memory - free: {}   allocated: {}]'.format(gc.mem_free(), gc.mem_alloc()))

## DSP processor

In [1]:
# from factory import Factory

with_hardware_device = True

if with_hardware_device:
    _i2c = peripherals.I2C.get_uPy_i2c(scl_pin_id = 18,
                                       sda_pin_id = 5)
else:
    _i2c = None  # using None for testing without actual hardware device.

bus = adapters.I2C(_i2c)

In [1]:
dsp = ADAU(bus)

## SigmaStudio project file

In [1]:
# SigmaStudio_project_file_url = os.sep.join(['..', '..', 'SigmaStudio projects', 'projects', 'demo_upy', 'demo_upy.dspproj'])

![](https://github.com/Wei1234c/SigmaDSP/blob/master/SigmaStudio%20projects/projects/demo_upy/demo_upy.png?raw=true)

## Factory

In [1]:
project_xml_file_url = 'demo_upy.xml'

factory = Factory(project_xml_file_url = project_xml_file_url,
                  dsp = dsp
                  )

print('factory ready.')

factory ready.


## IC

In [1]:
# showing cells, algorithms, parameters of this project

ic = factory.get_ic()

## Cells 

In [1]:
cells = factory.get_cells(ic)
print(cells.keys())

dict_keys(['Source_Switch', 'Gain_0', 'FIR0', 'FIR1', 'Tone1', 'Tone0', 'Mute_1', 'Mute_0', 'Gain_1', 'White_Noise'])


### Get Cell objects ready for use.

In [1]:
# un-comment this to generate a script to embody Cell objects.

# for o in factory.get_cells_manifest(ic):
#     print(o) 

In [1]:
# assign variable names to each Cell for easy access.

fir0 = cells['FIR0']  # FIR( 1 )
fir1 = cells['FIR1']  # FIR( 1 )
gain_0 = cells['Gain_0']  # Gain (no slew)( 1 )
gain_1 = cells['Gain_1']  # Gain (no slew)( 1 )
mute_0 = cells['Mute_0']  # No Slew (Standard)( 1 )
mute_1 = cells['Mute_1']  # No Slew (Standard)( 1 )
source_switch = cells['Source_Switch']  # Stereo SW Slew( 3 )
tone0 = cells['Tone0']  # Tone Synthesis (lookup/sine)( 1 )
tone1 = cells['Tone1']  # Tone Synthesis (lookup/sine)( 1 )
white_noise = cells['White_Noise']  # White Noise( 1 )

## Functional Demostration

In [1]:
# switch to sine tone

source_switch.switch(0)

In [1]:
# get cell by name

tone1 = factory.get_cell_by_name('Tone1', ic)

In [1]:
# disable sine tones

tone0.enable(False)
tone1.enable(False)

In [1]:
# enable sine tones

tone0.enable(True)
tone1.enable(True)

In [1]:
# set frequency 

tone0.set_frequency(440)
tone1.set_frequency(436)

In [1]:
# set frequency 

tone0.set_frequency(1100)
tone1.set_frequency(1104)

In [1]:
# set frequency 

tone0.set_frequency(110)
tone1.set_frequency(114)

In [1]:
# switch to white noise

source_switch.switch(1)

In [1]:
# decrease volume

gain_0.set_dB(-20)

In [1]:
# increase volume

gain_0.set_dB(0)

In [1]:
# mute input

mute_0.mute(True)
mute_1.mute(True)

In [1]:
# un-mute input

mute_0.mute(False)
mute_1.mute(False)

In [1]:
# disable white noise

white_noise.enable(False)

In [1]:
# enable white noise

white_noise.enable(True)

In [1]:
# set input0 dB = 0

gain_0.set_dB(0)

In [1]:
# set input0 dB = -10

gain_0.set_dB(-10)

In [1]:
# read input0 dB value

print(gain_0.get_dB())

-10.0


In [1]:
# read input0 gain value

print(gain_0.get_gain())

0.3162277


In [1]:
# set input0 gain = 1

gain_0.set_gain(1)

In [1]:
# check input0 dB value

print(gain_0.get_dB())

0.0


In [1]:
# check input0 gain value

print(gain_0.get_gain())

1.0


In [1]:
# read FIR0's coefficients

coeffs = fir0.get_coefficients()
print(coeffs)

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [1]:
# set and check FIR0's coefficients

fir0.set_coefficients([1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0])
print(fir0.get_coefficients())

[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]


In [1]:
# coefficients of FIR0 can also be accessed via. get_table() method.

print(fir0.get_table().numbers)

[1.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0]


In [1]:
# reload from EEPROM, equivalent to soft-reset

dsp.reload_from_eeprom()

## Testing Video

[![Testing Video](https://raw.githubusercontent.com/Wei1234c/SigmaDSP/master/jpgs/demo_upy3.jpg)](https://youtu.be/R3jcmxOhpao)     

### Note:
Needs ESP32 with more RAM space, usually with 8MB PSRAM.